plot_change.py: Generate significance of electrode weight changes at neuroanatomical sites

## Pseudocode:
    . Import
    . Input symptom, mode, perdur
    . Define functions
    . Load Parameters
    . For Loop through seizures

## Input symptom, mode, and duration

In [1]:
# sx_input = 'rex'
# mx_input = '1'
# perdur_input = '15'

## Import

In [2]:
import csv
import os
import pathlib
import xlsxwriter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.io import loadmat
from scipy.signal import butter,filtfilt
from scipy.stats import ttest_1samp

In [3]:
neuroanat_list = ['frontalpole', #FRONTAL LOBE
    'parstriangularis',
    'parsopercularis',
    'parsorbitalis',
    'rostralmiddlefrontal',
    'caudalmiddlefrontal',
    'lateralorbitofrontal',
    'superiorfrontal',
    'medialorbitofrontal',
    'precentral',
    'postcentral', # PARIETAL LOBE
    'inferiorparietal',   
    'superiorparietal',
    'supramarginal',
    'temporalpole', # TEMPORAL LOBE
    'middletemporal',
    'superiortemporal',
    'inferiortemporal',
    'parahippocampal',               
    'Right-Hippocampus',
    'Left-Hippocampus',
    'Right-Amygdala',
    'Left-Amygdala',
    'entorhinal',
    'bankssts',
    'fusiform', # OCCIPITAL LOBE                
    'lingual']
    # 'Right-Inf-Lat-Vent', # OTHER
    # 'Right-Cerebral-White-Matter',
    # 'Left-Cerebral-White-Matter',
    # 'Right-choroid-plexus',
    # 'Right-Putamen',
    # 'Right-VentralDC'];
    
abv_neuroanat_list = ['front-pole', #FRONTAL LOBE
    'parstri',
    'parsop',
    'parsorb',
    'rost-midfront',
    'caud-midfront',
    'latorb-front',
    'sup-front',
    'medorb-front',
    'precentral',
    'postcentral', # PARIETAL LOBE
    'inf-par',   
    'sup-par',
    'supra-marg',
    'temp-pole', # TEMPORAL LOBE
    'mid-temp',
    'sup-temp',
    'inf-temp',
    'parahip',               
    'R-Hip',
    'L-Hip',
    'R-Amyg',
    'L-Amyg',
    'entorhinal',
    'bankssts',
    'fusiform', # OCCIPITAL LOBE                
    'lingual']
    # 'Right-Inf-Lat-Vent', # OTHER
    # 'Right-Cerebral-White-Matter',
    # 'Left-Cerebral-White-Matter',
    # 'Right-choroid-plexus',
    # 'Right-Putamen',
    # 'Right-VentralDC']
# sx_list = ['lex',
#            'lhx',
#            'lnx',
#            'lfx',
#            'ltx',
#            'lmx',
#            'lud',
#            'lup',
#            'lld',
#            'llp',
#            'rex',
#            'rhx',
#            'rnx',
#            'rfx',
#            'rtx',
#            'rmx',
#            'rud',
#            'rup',
#            'rld',
#            'rlp',
#            'brx',
#            'fax',
#            'oax',
#            'qcx']
sx_list = ['lhx']
mx_list = ['2']
# mx_list = ['1','2','3']
perdur_input = 15

## Define Functions

In [4]:
def sem_w8s(LL,at_onset,before_onset,after_onset):
    row_num = np.shape(LL)[0]
    LL_meandiff = np.empty(row_num,)
    for row in range(0,row_num-1): #for each channel in LL   
         LL_meandiff[row] = np.mean(LL[row,int(at_onset):int(after_onset)]) - np.mean(LL[row,int(before_onset):int(at_onset)])
    return LL_meandiff

In [5]:
def ll_transform(llw,fs,d,bl_start,bl_stop):    
    if bl_start == 0:
        sample_bl_start = 0
    elif bl_start > 0:
        sample_bl_start = (fs*bl_start)[0] #0 because it's python not matlab
    sample_bl_end = (fs*bl_stop)[0]-1
    
    L = int(np.round(llw * fs) - 1) # number of samples to calculate line length
    
    col_len = np.shape(d)[1]-L
    
    LL = np.empty([np.shape(d)[0],col_len])
    
    LL[:] = np.NaN        
    
    for col_1 in range(0,col_len):
        LL[:,col_1] = np.sum(np.abs(np.diff(d[:,col_1:col_1+L])),1)


    for row_1 in range(0,np.shape(d)[0]):    
        LL_nanmean = np.nanmean(LL[row_1,sample_bl_start:sample_bl_end])
        LL_nanstd = np.nanstd(LL[row_1,sample_bl_start:sample_bl_end])
                                              
        LL[row_1,:] = (LL[row_1,:] - LL_nanmean)/LL_nanstd
                    
    return LL


In [6]:
def open_xl(xl_name): 
    
    df = pd.read_excel(xl_name, index_col=0, engine='openpyxl')

    return df


In [7]:
def filt(d,fs):
    d_t = d.transpose()
    
    butter_array = np.array([1,(round(fs[0]/2)-1)])
    b,a = butter(2,butter_array/(fs[0]/2),btype='bandpass',output='ba')
    
    filt_d = filtfilt(b,a,d_t,axis=0,padtype='odd',padlen=3*(max(len(b),len(a))-1)).transpose()

    return filt_d


In [8]:
def load_elecs_anat(pt_path):
                
    #load anatomy and electrode matrix
    os.chdir(pt_path + 'Imaging/elecs')
    e_mat = loadmat('clinical_elecs_all.mat')
    anat_col = e_mat['anatomy'][:,3]
            
    return anat_col


In [9]:
def get_params(df_params,pt):
    params_bl_start = df_params.loc[pt]['BLstart']
    params_bl_stop = df_params.loc[pt]['BLstop']
    params_llw = df_params.loc[pt]['llw']

    return params_bl_start, params_bl_stop, params_llw

In [10]:
def get_ptsz(avg_change_path): #Load list of patient and seizure names
    os.chdir(avg_change_path)
    
    df_params = open_xl('OPSCEAparams.xlsx') # Ndimensions and params_list

    with open('sz_name_list.csv','r') as sz_names:
        sz_names_list = list(csv.reader(sz_names))
        sz_rows = np.size(sz_names_list,0) #find row num 
        pt_data = []
        sz_data= []
        for r in range(0,sz_rows):  #collect symptom data for specfic seizure in list
            pt_data.append(sz_names_list[r][0]) 
            sz_data.append(sz_names_list[r][1]) 
    return pt_data, sz_data, df_params

## For loop through patients in list matching seizure

In [11]:
# def neurosem(sx_input,mx_input,perdur_input,pt_data,sz_data,df_params,workbook):
avg_change_path = '/Users/nataliasucher/Desktop/UCSF/Coding/OPSCEA/OPSCEADATA/avg_change_folders/'

pt_data,sz_data,df_params = get_ptsz(avg_change_path) #Ehsan

# create xlsx workbook 
workbook = xlsxwriter.Workbook('Neurosemiology.xlsx')
dark_green_format = workbook.add_format({'bg_color': '#30B700'})
light_green_format = workbook.add_format({'bg_color': '#98FB98'})
yellow_format = workbook.add_format({'bg_color': '#FFE900'})
red_format = workbook.add_format({'bg_color': '#FF7276'})
white_format = workbook.add_format({'bg_color': '#FFFFFF'})



#For loop through patients
for i in range(len(pt_data)):  #Ehsan
    pt_i = pt_data[i]          #Ehsan
    sz_name = sz_data[i]       #Ehsan

    pt_path = avg_change_path + pt_i + '/'
    os.chdir(pt_path)
    pt_dir = os.listdir(pt_path)

    for sz_name in sz_data: 

        if sz_name in pt_dir: #patients in list matching seizure
            print(sz_name)
            # PARAMS
            blstart, blstop, llw = get_params(df_params,pt_i) # 2 = params_llw

            # ANAT 
            anat = load_elecs_anat(pt_path)


            # SEMIOLOGY
            sz_path = pt_path + sz_data[i]
            os.chdir(sz_path)

            for sx_input in sx_list:
                for mx_input in mx_list:
                    # XLSX

                    worksheet_name = sx_input+' '+mx_input
                    if i == 0:
                        # add 1 xlsx worksheet per symptom/mode
                        worksheet = workbook.add_worksheet(worksheet_name) 
                        worksheet.write(0,0,worksheet_name)
                        worksheet.write(0, i+1, sz_data[i])

                        print('0: '+worksheet_name)
                    elif i > 0:
                        print(worksheet_name)

                        worksheet = workbook.get_worksheet_by_name(worksheet_name)
                        worksheet.write(0, i+1, sz_data[i])            




                    sx_vec = pd.read_csv(sz_name + '_mat.csv',usecols = [sx_input]).values.tolist() #load semiology matrix .csv             #WANT TO JUST LOAD 1 COLUMN SX_INPUT            

                    if np.float64(mx_input) in sx_vec:

                        sz_mat = loadmat(sz_name +'.mat') #load frame speed and ppEEG
                        fs = sz_mat['fs'].flatten()
                        d = sz_mat['ppEEG'][0:np.shape(anat)[0],:]

                        badch_mat = loadmat(sz_name + '_badch.mat')
                        badch = badch_mat['bad_chs']

                        badch = np.delete(badch,range(np.shape(anat)[0],np.shape(badch)[0]),0)

                        for a_i in range(0,np.shape(anat)[0]):
                            if np.size(anat[a_i]) > 0: 
                                if anat[a_i][0] not in neuroanat_list:
                                    badch[a_i][0] = 1 #prepare for anatomy not in neuroanat list to be deleted in d and anat

                        bad_logical = np.any(badch,1)
                        d = d[~bad_logical,:]
                        anat = anat[~bad_logical]

                        filt_d = filt(d,fs) #Filter out < 1 Hz (and up to nyquist)

                        LL = ll_transform(llw,fs,filt_d,blstart,blstop) 

                        at_onset = np.round(((sx_vec.index(np.float64(mx_input)))/5) * fs)
                        before_onset = np.round(((sx_vec.index(np.float64(mx_input))/5) - float(perdur_input)) * fs)
                        after_onset = np.round(((sx_vec.index(np.float64(mx_input))/5) + float(perdur_input)) * fs)
                        
                        print('before ', before_onset, LL[0,int(before_onset):int(at_onset)])

                        print('after: ',after_onset, LL[0,int(at_onset):int(after_onset)])

                        LL_meandiff = sem_w8s(LL,at_onset,before_onset,after_onset)

                        pval = np.empty(np.shape(neuroanat_list))
                        pval[:] = np.NaN

                        #string comparison 
                        anat_index = [np.NaN] * np.shape(anat)[0]


                        anat_list = np.empty(np.shape(anat)).tolist()
                        anat_w8s = []
                        for a in range(0,np.shape(anat)[0]):
                            if len(anat[a]) > 0:
                                anat_list[a] = anat[a][0]


                        for n_l in range(0,len(neuroanat_list)):
                            worksheet.write(n_l+1,0,neuroanat_list[n_l])
                            #clump together all the ones that match same neuroanat
                            for a_i in range(0,len(anat_list)):
                                if neuroanat_list[n_l] == anat_list[a_i][:]:
                                    anat_w8s.append(LL_meandiff[a_i])
                            # print(anat_w8s)
                            tstat, pval[n_l] = ttest_1samp(anat_w8s,0)

                            try: 
                                worksheet.write(n_l+1,i+1,pval[n_l])
                            except TypeError:
                                pass
                                    # print(pval[n_l])

                            # print(neuroanat_list[n_l],': ',pval[n_l])

                            anat_w8s = []

                        # worksheet.conditional_format('B2:H28',{'type':'cell',
                        #                                        'criteria':'between',
                        #                                        'minimum':.001,
                        #                                        'maximum':.08,
                        #                                        'format': light_green_format})
                        # worksheet.conditional_format('B2:H28',{'type':'cell',
                        #                                        'criteria':'between',
                        #                                        'minimum':.08,
                        #                                        'maximum':.25,
                        #                                        'format': yellow_format})
                        # worksheet.conditional_format('B2:H28',{'type':'cell',
                        #                                        'criteria':'between',
                        #                                        'minimum': .25,
                        #                                        'maximum':1,
                        #                                        'format': red_format})
                        # worksheet.conditional_format('B2:H28',{'type':'cell',
                        #                                        'criteria':'between',
                        #                                        'minimum':.00000000000000000000000000001,
                        #                                        'maximum':.001,
                        #                                        'format': dark_green_format})
                        # #WHEN CELL = 0
                        # worksheet.conditional_format('B2:H28',{'type':'blanks',
                        #                                        'format': white_format})
                        # worksheet.conditional_format('B2:H28',{'type':'cell',
                        #                                        'criteria':'equal to',
                        #                                        'value':0,
                        #                                        'format': white_format})


                        
workbook.close()

In [15]:
tstat, pval = ttest_1samp([-33.00863798812543, -7.4380449766326855],0)
print(pval)


0.35890307439208957


EC91_03
0: lhx 2
before  [78746.] [99.6427128  99.5747957  99.5177878  ... 44.64605059 44.62662894
 44.59740051]
after:  [94106.] [44.58003305 44.52605311 44.46250847 ... 22.63842685 22.67124712
 23.04131195]
[]
[43.753066481102756, 37.949620367933974, 23.683676203300227, -0.8057216399810088]
[32.37279887692469, 4.701256375897584, 21.536496820927375]
[-6.873176212912805, -3.4265878922217894]
[1.6656225510354457, 7.616847702911814, 2.512989294147875]
[-2.12009100805955, 27.2331061556729]
[-9.541887439999819, -17.027906818223148, -8.547816600328298]
[]
[]
[51.17685088241599, 38.76083773653195, 12.814532715977075, 16.44632778131463, 34.29807959258908, 15.675963263423359, 33.94729352569499, 36.22931819752382, 37.960202634342494]
[18.019787184975215, 34.32925689450076, 3.8248791931331976, 19.771438650821132, 20.201493690964853, 9.788858990491544, 3.251319634097554, 0.6898728558111926]
[]
[]
[30.54181201270463, 2.8776235106141854, 4.887065413279041, 2.372288176631006, 0.9281174267178671, 3.3

/Users/nataliasucher/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/nataliasucher/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/nataliasucher/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3702: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/nataliasucher/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/nataliasucher/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


before  [41779.] [13.09815415 13.08234183 13.34877118 ... 47.53837716 47.45411172
 47.33226423]
after:  [57139.] [47.20051591 47.08617255 46.8469612  ... 33.76736979 33.91180362
 34.34356675]
[]
[4.14960695531062, 14.065158953211046]
[9.419279258195498, 12.586867601985809, 15.168602792019906, 8.52360559979568, 7.990306735278897, 2.615543108990908]
[-0.9552281934318856, 0.7856739439412337]
[1.2425061949943446, 1.5920701873931837]
[8.949728028761033, 2.6916822159071128]
[4.918233105324783, 2.649304021700713, 0.6616537917169341, 3.8431247005338225]
[]
[]
[22.86643487332467, 13.189833571736585, 12.37161132130174, 15.747167835433752, 4.3453956479713245, 8.741708325472532, 25.339929215637916, 19.178233221812395, 22.528378847160194, 17.05492607397629]
[7.352386031765814, 17.56107029438344, 15.242789236604748, 12.419712098832454, 11.726980234307963, 6.0951894709537555, 11.894920230020121, 6.6827338340449565, 5.234569357792987, 7.52418702698281, 3.6544066466189062, 2.794038844416505, 0.63692911